In [1]:
#Still errors in this model
import os

from keras.callbacks import ModelCheckpoint

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import glob
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.sequence import pad_sequences
from numpy import genfromtxt
from keras.utils import np_utils
import tensorflow
import numpy
import os
import cv2
import sys
from keras import backend as K
K.set_image_dim_ordering('th')

cwd = os.getcwd()

Using TensorFlow backend.


In [2]:
#Part 1 -> defines
img_rows, img_cols = 256, 256
nb_filters = 32 # nr of conv filters to use
nb_pool = 2 # size of pooling area
nb_conv = 3 # convolution kernel size

In [3]:
#Part 2 -> get data
def getData():
    train_datagen = ImageDataGenerator(
        rotation_range = 20,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
    )

    test_datagen = ImageDataGenerator(rescale = 1./255)

    training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                     target_size = (img_rows, img_cols),
                                                     batch_size = 64,
                                                     class_mode = 'sparse',
                                                     color_mode = 'rgb')

    test_set = train_datagen.flow_from_directory('dataset/test_set',
                                                 target_size = (img_rows, img_cols),
                                                 batch_size = 64,
                                                 class_mode = 'sparse',
                                                 color_mode = 'rgb')

    return training_set, test_set

In [4]:
#Part 3 -> get the model
def getModel():
    classifier = Sequential()

    # Step 1 - Convolution
    classifier.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                                border_mode = 'valid',
                                input_shape=(3, img_rows, img_cols),
                                activation = 'relu'))

    # Step 2 - Pooling
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    # Adding a second convolutional layer
    classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    # Step 5 -> Flattening
    classifier.add(Flatten())

    # Step 6 -> Full connection
    classifier.add(RepeatVector(3))
    classifier.add(Reshape( (3,-1) ))
    classifier.add(LSTM(6, return_sequences = True ))
    
    #classifier.add(TimeDistributed( Dense(output_dim = 256, activation = 'relu') ))
    #classifier.add(TimeDistributed( Dropout(0.2) ))
    #classifier.add(TimeDistributed( Dense(output_dim = 256, activation = 'relu') ))
    #classifier.add(TimeDistributed( Dense(output_dim = 1) ))
    
    
    #Part 3 -> Compile the CNN
    classifier.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
    
    print(classifier.summary())
    return classifier

In [5]:
#Part 4 -> fit the model
def fitModel(training_set, test_set, classifier):
    filepath = sys.argv[0] + "-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    classifier.fit_generator(training_set,
                             samples_per_epoch = 8000,
                             nb_epoch = 120,
                             validation_data = test_set,
                             nb_val_samples = 3968,
                             callbacks = callbacks_list)

In [6]:
#Part 5 -> run everything
training_set, test_set = getData()
classifier = getModel()
fitModel(training_set, test_set, classifier)

Found 100000 images belonging to 900 classes.
Found 50000 images belonging to 900 classes.


TypeError: Expected binary or unicode string, got -1